In [1]:
import pandas as pd
import nltk
import enchant
import re
from nltk.metrics import edit_distance
import spacy
import tensorflow as tf
tf.get_logger().setLevel('INFO')
from nltk import word_tokenize
from fastai.text import *
from nltk.corpus import stopwords
stop=set(stopwords.words('english'))
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam

C:\Users\Olek\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Olek\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Olek\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Olek\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:5

In [2]:
test = pd.read_csv('Disaster Tweets/test.csv')

In [3]:
train = pd.read_csv('Disaster Tweets/train.csv')
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


__Columns__<br>
- __id__ - a unique identifier for each tweet
- __text__ - the text of the tweet
- __location__ - the location the tweet was sent from (may be blank)
- __keyword__ - a particular keyword from the tweet (may be blank)
- __target__ - in train.csv only, this denotes whether a tweet is about a real disaster (1) or not (0)

In [4]:
train['keyword'].value_counts()

fatalities               45
armageddon               42
deluge                   42
body%20bags              41
damage                   41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: keyword, Length: 221, dtype: int64

In [5]:
train['location'].value_counts()

USA                   104
New York               71
United States          50
London                 45
Canada                 29
                     ... 
ELVY                    1
Nairobi, Kenya          1
The Internet & NYC      1
London.                 1
Tucson, Az              1
Name: location, Length: 3341, dtype: int64

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [7]:
replacement_patterns = [
    (r'won\'t', 'will not'),
    (r'can\'t', 'cannot'),
    (r'i\'m', 'i am'),
    (r'ain\'t', 'is not'),
    (r'(\w+)\'ll', '\g<1> will'),
    (r'(\w+)n\'t', '\g<1> not'),
    (r'(\w+)\'ve', '\g<1> have'),
    (r'(\w+)\'s', '\g<1> is'),
    (r'(\w+)\'re', '\g<1> are'),
    (r'(\w+)\'d', '\g<1> would'),
]

class RegexpReplacer(object):
    # Replaces regular expression in a text.
    def __init__(self, patterns=replacement_patterns):
        self.patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]
    
    def replace(self, text):
        s = text
        
        for (pattern, repl) in self.patterns:
            s = re.sub(pattern, repl, s)
        
        return s

class SpellingReplacer(object):
    """ Replaces misspelled words with a likely suggestion based on shortest
    edit distance
    """
    def __init__(self, dict_name='en', max_dist=2):
        self.spell_dict = enchant.Dict(dict_name)
        self.max_dist = max_dist
    
    def replace(self, word):
        if self.spell_dict.check(word):
            return word
        
        suggestions = self.spell_dict.suggest(word)
        
        if suggestions and edit_distance(word, suggestions[0]) <= self.max_dist:
            return suggestions[0]
        else:
            return word

In [8]:
ids_with_target_error = [328, 443, 513, 2619, 3640, 3900, 4342, 5781, 6552, 6554, 6570, 6701, 6702, 6729, 6861, 7226]
incorrect = train[train['id'].isin(ids_with_target_error)]
incorrect

,id,keyword,location,text,target
229,328,annihilated,NaN,Ready to get annihilated for the BUCS game,1
301,443,apocalypse,NaN,Short Reading\n\nApocalypse 21:1023 \n\nIn the...,1
356,513,army,Studio,But if you build an army of 100 dogs and their...,1
1822,2619,crashed,NaN,My iPod crashed..... \n#WeLoveYouLouis \n#MTVH...,1
2536,3640,desolation,"Quilmes , Arg",This desperation dislocation\nSeparation conde...,1
2715,3900,devastated,PG Chillin!,Man Currensy really be talkin that talk... I'd...,1
3024,4342,dust%20storm,chicago,Going to a fest? Bring swimming goggles for th...,1
4068,5781,forest%20fires,NaN,Campsite recommendations \nToilets /shower \nP...,1
4609,6552,injury,Saint Paul,My prediction for the Vikings game this Sunday...,1
4611,6554,injury,NaN,Dante Exum's knee injury could stem Jazz's hop...,1


In [9]:
train.loc[train['id'].isin(ids_with_target_error), 'target'] = 0
train[train['id'].isin(ids_with_target_error)]

,id,keyword,location,text,target
229,328,annihilated,NaN,Ready to get annihilated for the BUCS game,0
301,443,apocalypse,NaN,Short Reading\n\nApocalypse 21:1023 \n\nIn the...,0
356,513,army,Studio,But if you build an army of 100 dogs and their...,0
1822,2619,crashed,NaN,My iPod crashed..... \n#WeLoveYouLouis \n#MTVH...,0
2536,3640,desolation,"Quilmes , Arg",This desperation dislocation\nSeparation conde...,0
2715,3900,devastated,PG Chillin!,Man Currensy really be talkin that talk... I'd...,0
3024,4342,dust%20storm,chicago,Going to a fest? Bring swimming goggles for th...,0
4068,5781,forest%20fires,NaN,Campsite recommendations \nToilets /shower \nP...,0
4609,6552,injury,Saint Paul,My prediction for the Vikings game this Sunday...,0
4611,6554,injury,NaN,Dante Exum's knee injury could stem Jazz's hop...,0


In [10]:
def clean_tweet(text) :
    # remove urls
    #text = df.apply(lambda x: re.sub(r'http\S+', '', x))
    text = re.sub(r'http\S+', '', text)

    # replace contractions
    replacer = RegexpReplacer()
    text = replacer.replace(text)

    #split words on - and \
    text = re.sub(r'\b', ' ', text)
    text = re.sub(r'-', ' ', text)

    # replace negations with antonyms

    #nltk.download('punkt')
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    tokens = tokenizer.tokenize(text)

    # spelling correction
    replacer = SpellingReplacer()
    tokens = [replacer.replace(t) for t in tokens]

    # lemmatize/stemming
    wnl = nltk.WordNetLemmatizer()
    tokens = [wnl.lemmatize(t) for t in tokens]
    porter = nltk.PorterStemmer()
    tokens = [porter.stem(t) for t in tokens]
    # filter insignificant words (using fastai)
    # swap word phrases

    text = ' '.join(tokens)
    return(text)

In [11]:
tweets = train['text']
tqdm.pandas(desc="Cleaning tweets")
tweets_cleaned = tweets.progress_apply(clean_tweet)
train['text'] = tweets_cleaned
train.head()

Cleaning tweets: 100%|██████████| 7613/7613 [11:19<00:00, 11.20it/s]


,id,keyword,location,text,target
0,1,NaN,NaN,our deed are the reason of thi earthquak may a...,1
1,4,NaN,NaN,forest fire near La long sask canada,1
2,5,NaN,NaN,all resid ask to shelter in place are be notif...,1
3,6,NaN,NaN,13 000 peopl receiv wildfir evacu order in cal...,1
4,7,NaN,NaN,just got sent thi photo from rubi alaska a smo...,1


In [12]:
tweets = test['text']
tqdm.pandas(desc="Cleaning tweets")
tweets_cleaned = tweets.progress_apply(clean_tweet)
test['text'] = tweets_cleaned
test.head()

Cleaning tweets: 100%|██████████| 3263/3263 [04:55<00:00, 11.02it/s]


,id,keyword,location,text
0,0,NaN,NaN,just happen a terribl car crash
1,2,NaN,NaN,heard about earthquak is differ citi stay safe...
2,3,NaN,NaN,there is a forest fire at spot pond goos are f...
3,9,NaN,NaN,apocalyps light spokan wildfir
4,11,NaN,NaN,typhoon soudelor kill 28 in china and taiwan


In [13]:
def create_corpus(df):
    corpus=[]
    for tweet in tqdm(df['text']):
        words=[word.lower() for word in word_tokenize(tweet) if((word.isalpha()==1) & (word not in stop))]
        corpus.append(words)
    return corpus

In [14]:
df = pd.concat([train, test])
corpus=create_corpus(df)

100%|██████████| 10876/10876 [00:00<00:00, 11528.61it/s]


In [15]:
embedding_dict={}
with open('D:/Notebooks/Disaster Tweets/glove.twitter.27B.100d.txt', 'r', encoding="utf8") as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:], 'float32')
        embedding_dict[word]=vectors
f.close()

In [16]:
MAX_LEN=50
tokenizer_obj=Tokenizer()
tokenizer_obj.fit_on_texts(corpus)
sequences=tokenizer_obj.texts_to_sequences(corpus)

tweet_pad=pad_sequences(sequences, maxlen=MAX_LEN, truncating='post', padding='post')

In [17]:
word_index=tokenizer_obj.word_index
print('Number of unique words:', len(word_index))

Number of unique words: 13557


In [18]:
num_words=len(word_index)+1
embedding_matrix=np.zeros((num_words,100))

for word,i in tqdm(word_index.items()):
    if i > num_words:
        continue
    
    emb_vec=embedding_dict.get(word)
    if emb_vec is not None:
        embedding_matrix[i]=emb_vec
            

100%|██████████| 13557/13557 [00:00<00:00, 803898.88it/s]


In [19]:
model=Sequential()

embedding=Embedding(num_words,100, embeddings_initializer=Constant(embedding_matrix),
                   input_length=MAX_LEN, trainable=False)

model.add(embedding)
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))


optimzer=Adam(learning_rate=1e-5)

model.compile(loss='binary_crossentropy', optimizer=optimzer, metrics=['accuracy'])

In [20]:
train_df=tweet_pad[:train.shape[0]]
val_df=tweet_pad[train.shape[0]:]

In [21]:
X_train, X_val, y_train, y_val=train_test_split(train_df, train['target'].values, test_size=0.15)
print('Shape of train', X_train.shape)
print("Shape of Validation ", X_val.shape)

Shape of train (6471, 50)
Shape of Validation  (1142, 50)


In [22]:
history = model.fit(X_train, y_train, batch_size=4, epochs=15, validation_data=(X_val, y_val), verbose=2)

Train on 6471 samples, validate on 1142 samples
Epoch 1/15
 - 134s - loss: 0.6916 - accuracy: 0.5722 - val_loss: 0.6893 - val_accuracy: 0.5797
Epoch 2/15
 - 100s - loss: 0.6829 - accuracy: 0.5733 - val_loss: 0.6548 - val_accuracy: 0.5814
Epoch 3/15
 - 100s - loss: 0.6208 - accuracy: 0.6704 - val_loss: 0.5509 - val_accuracy: 0.7671
Epoch 4/15
 - 99s - loss: 0.5933 - accuracy: 0.7016 - val_loss: 0.5322 - val_accuracy: 0.7715
Epoch 5/15
 - 100s - loss: 0.5928 - accuracy: 0.7079 - val_loss: 0.5236 - val_accuracy: 0.7723
Epoch 6/15
 - 100s - loss: 0.5851 - accuracy: 0.7181 - val_loss: 0.5171 - val_accuracy: 0.7741
Epoch 7/15
 - 102s - loss: 0.5795 - accuracy: 0.7178 - val_loss: 0.5128 - val_accuracy: 0.7776
Epoch 8/15
 - 100s - loss: 0.5711 - accuracy: 0.7277 - val_loss: 0.5087 - val_accuracy: 0.7793
Epoch 9/15
 - 101s - loss: 0.5743 - accuracy: 0.7255 - val_loss: 0.5064 - val_accuracy: 0.7802
Epoch 10/15
 - 101s - loss: 0.5769 - accuracy: 0.7226 - val_loss: 0.5055 - val_accuracy: 0.7855
Ep

In [23]:
sample_sub=pd.read_csv('Disaster Tweets/sample_submission.csv')

In [27]:
y_pre=model.predict(val_df)
y_pre=np.round(y_pre).astype(int).reshape(3263)
sub=pd.DataFrame({'id':sample_sub['id'].values.tolist(),'target':y_pre})
sub.to_csv('Disaster Tweets/submission.csv',index=False)